# Tuto to find The sources and Visits and Objects in LSSTComCam

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-03
- last update : 2025-05-03

## Note on work done on LSSTComCam Commissioning : https://sitcomtn-149.lsst.io/
## Note Data Product definition Document : https://lse-163.lsst.io/

In [ ]:
import sys, os
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import numpy as np
import pandas as pd
from astropy.time import Time
# %matplotlib widget

In [ ]:
import lsst.geom as geom
from lsst.geom import SpherePoint, degrees, Point2D, Point2I
from lsst.skymap import PatchInfo, Index2D

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

In [ ]:
import gc

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

In [ ]:
# def nJy_to_ab_mag(f_njy):
#    """Convert flux in nanojanskys to AB magnitude."""
#    if f_njy > 0:
#        return -2.5 * np.log10(f_njy) + 31.4
#    else:
#        return 0.0


def ab_mag_to_nJy(mag_ab):
    """Convert AB magnitude to flux in nanojanskys."""
    return 10 ** ((31.4 - mag_ab) / 2.5)

In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err

In [ ]:
def select_visits_in_patch(df, tractInfo, patchInfo):
    """
    Sélectionne les lignes du DataFrame correspondant à des visites dans un patch donné.

    Paramètres
    ----------
    df : pandas.DataFrame
        Doit contenir les colonnes 'ra' et 'dec' en degrés.
    tractInfo : lsst.skymap.TractInfo
    patchInfo : lsst.skymap.PatchInfo

    Retourne
    --------
    df_filtered : pandas.DataFrame
        Les lignes du DataFrame dont les RA/Dec sont dans le patch spécifié.
    """
    wcs = tractInfo.getWcs()
    bbox = patchInfo.getOuterBBox()

    def is_inside_patch(ra, dec):
        coord = SpherePoint(ra * degrees, dec * degrees)
        if not tractInfo.contains(coord):
            return False
        pix = wcs.skyToPixel(coord)
        return bbox.contains(Point2I(pix))

    mask = df.apply(lambda row: is_inside_patch(row["ra"], row["dec"]), axis=1)
    return df[mask].copy()

In [ ]:
# Tested but long
def assign_tract_patch(df, skyMap):
    """
    Attribue (tract, patch) à chaque ligne d’un DataFrame selon ses coordonnées célestes.

    Paramètres
    ----------
    df : pandas.DataFrame
        Doit contenir les colonnes 'ra' et 'dec' (en degrés).
    skyMap : lsst.skymap.SkyMap
        Obtenu via butler.get("skyMap")

    Retour
    ------
    df_result : pandas.DataFrame
        DataFrame original avec deux colonnes supplémentaires :
        'tract' et 'patch' (tuple (x, y) ou None si hors carte).
    """

    def find_tract_patch(ra, dec):
        coord = SpherePoint(ra * degrees, dec * degrees)
        for tractInfo in skyMap:
            if tractInfo.contains(coord):
                wcs = tractInfo.getWcs()
                pix = wcs.skyToPixel(coord)
                point = Point2I(pix)
                for patchInfo in tractInfo:
                    if patchInfo.getOuterBBox().contains(point):
                        return tractInfo.getId(), patchInfo.getSequentialIndex()
        return None, None

    results = df.apply(lambda row: find_tract_patch(row["ra"], row["dec"]), axis=1)
    df[["tract", "patch"]] = pd.DataFrame(results.tolist(), index=df.index)
    return df

In [ ]:
# Not tested
def filter_in_patch_vectorized(df, tractInfo, patchInfo):
    """
    Sélection rapide (vectorisée) d’un sous-ensemble RA/Dec inclus dans un patch donné.

    Attention : plus rapide, mais approximatif si les coordonnées sont très en bordure.

    Paramètres
    ----------
    df : pandas.DataFrame avec colonnes 'ra', 'dec'
    tractInfo : TractInfo
    patchInfo : PatchInfo

    Retour
    ------
    df_filtered : pandas.DataFrame
    """
    wcs = tractInfo.getWcs()
    bbox = patchInfo.getOuterBBox()

    # Conversion RA/Dec → radians
    ra_rad = np.deg2rad(df["ra"].values)
    dec_rad = np.deg2rad(df["dec"].values)

    # Conversion → Point2D (skyToPixel produit Point2D, pas Point2I)
    pix_x = []
    pix_y = []
    for ra, dec in zip(ra_rad, dec_rad):
        sp = SpherePoint(ra, dec)
        pix = wcs.skyToPixel(sp)
        pix_x.append(pix.getX())
        pix_y.append(pix.getY())

    pix_x = np.array(pix_x)
    pix_y = np.array(pix_y)

    # BBox (xmin, xmax, ymin, ymax)
    xmin, xmax = bbox.getMinX(), bbox.getMaxX()
    ymin, ymax = bbox.getMinY(), bbox.getMaxY()

    # Masque vectoriel
    mask = (pix_x >= xmin) & (pix_x <= xmax) & (pix_y >= ymin) & (pix_y <= ymax)

    return df[mask].copy()

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = True
FLAG_DUMP_OBJECTSTABLECOLUMNS = False
FLAG_DUMP_VISITTABLEECOLUMNS = True
FLAG_COMPUTE_VISITSTRACTSPATCHS = False

In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]

### Choose instrument

In [ ]:
# instrument = "LSSTCam"

# We focus here on sky fields oberved by LSSTComCam, so we select this camera
instrument = "LSSTComCam"

### Choose options

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_instrument = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"
    BANDSEL = "i"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work 2025-05-01
    # collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530" #updated 2025-05-02
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_instrument = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063
    BANDSEL = "i"

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object and Camera

In [ ]:
skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
camera = butler.get("camera", collections=collection_validation, instrument=instrument)

## Query for collections in Butler

- remove user collections
- remove calibration products

In [ ]:
# mostly setup for LSSTCam
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

## Query for the dataset types in the Butler

- Refer to the Data Product definition Document to know about the definition of datasets
- https://www.lsst.org/about/dm/data-products
- https://lse-163.lsst.io/
- https://docushare.lsst.org/docushare/dsweb/Get/LSE-163

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
            ):
                if "object" in datasetType.name or "Obj" in datasetType.name:
                    print(datasetType)
                if "source" in datasetType.name or "Source" in datasetType.name:
                    print(datasetType)
                if "visit" in datasetType.name or "Visit" in datasetType.name:
                    print(datasetType)

## Region of interest

In [ ]:
lsstcomcam_targets = {}
lsstcomcam_targets["47 Tuc"] = {"field_name": "47 Tuc Globular Cluster", "ra": 6.02, "dec": -72.08}
lsstcomcam_targets["Rubin SV 38 7"] = {"field_name": "Low Ecliptic Latitude Field", "ra": 37.86, "dec": 6.98}
lsstcomcam_targets["Fornax dSph"] = {
    "field_name": "Fornax Dwarf Spheroidal Galaxy",
    "ra": 40.0,
    "dec": -34.45,
}
lsstcomcam_targets["ECDFS"] = {"field_name": "Extended Chandra Deep Field South", "ra": 53.13, "dec": -28.10}
lsstcomcam_targets["EDFS"] = {"field_name": "Euclid Deep Field South", "ra": 59.10, "dec": -48.73}
lsstcomcam_targets["Rubin SV 95 -25"] = {
    "field_name": "Low Galactic Latitude Field",
    "ra": 95.00,
    "dec": -25.0,
}
lsstcomcam_targets["Seagull"] = {"field_name": "Seagull Nebula", "ra": 106.23, "dec": -10.51}

In [ ]:
# the_target = lsstcomcam_targets["Seagull"]
# the_target = lsstcomcam_targets["47 Tuc"] # bad
# the_target = lsstcomcam_targets["Fornax dSph"]
# the_target = lsstcomcam_targets["ECDFS"]


# key = "Seagull"
# key = "Fornax dSph"
key = "ECDFS"
# key = "EDFS"
# key = "47 Tuc"
# key = "Rubin SV 38 7"
# key = "Rubin SV 95 -25"

keyfield = "".join(key.split())

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_title = (
    the_target["field_name"] + f" band  {BANDSEL} " + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)
target_point = SpherePoint(target_ra, target_dec, degrees)

## Get list of tracts from the objectTable_tract

In [ ]:
# Find the dimension
print(butler.registry.getDatasetType("objectTable_tract").dimensions)

In [ ]:
datasettype = "objectTable_tract"
therefs = butler.registry.queryDatasets(datasettype, collections=collection_validation)
tractsId_list = np.unique([ref.dataId["tract"] for ref in therefs])
tractsId_list = sorted(tractsId_list)
print(tractsId_list)

## Find the Tract and Patch of the region of interest

- tract in tractNbSel
- patch in patchNbSel

In [ ]:
tract_info = skymap.findTract(target_point)
patch_info = tract_info.findPatch(target_point)
bbox = patch_info.getOuterBBox()

print("Patch bounding box:", bbox)

print("Tract ID :", tract_info.getId())
tractNbSel = tract_info.getId()

print("Patch Index :", patch_info.getIndex(), " , ", patch_info.getSequentialIndex())  # (x, y)
print("Bounding Box", bbox)

patchNbSel = patch_info.getSequentialIndex()

In [ ]:
dataId = {"band": BANDSEL, "tract": tractNbSel, "patch": patchNbSel, "skymap": skymapName}

## The Objects

- Objects are extracted from object detection on deepcoadds
- all bands are included 

In [ ]:
print(butler.registry.getDatasetType("objectTable").dimensions)

In [ ]:
# cannot add a filter on band
where_clause_skymap = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
print(where_clause)

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets("objectTable", collections=collection_validation, where=where_clause_skymap)
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of objectTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)
Nobj = len(t)
# del t
# gc.collect()
print(f"Total Number of objects {Nobj}")

Oui, dans ce type de objectTable, les colonnes comme g_psfFlux, g_kronFlux, g_cModelFlux, etc., sont des flux calibrés (en nJy ou en unité de calibration interne du pipeline). Pour les convertir en magnitudes AB, tu peux utiliser la formule classique :

In [ ]:
# Utilise ref.datasetType.name pour lister les colonnes disponibles
if FLAG_DUMP_OBJECTSTABLECOLUMNS:
    t_columns = list(butler.get(ref).columns)
    print(t_columns)

In [ ]:
# La constante 31.4 correspond à la conversion AB standard pour un flux exprimé en nanoJanskys (nJy).
# À vérifier selon l’unité exacte utilisée par le pipeline sur ton RSP
# (souvent c’est bien nJy, mais ça peut être autre chose si la calibration a été changée).
# mag = -2.5 * np.log10(flux) + 31.4

## Visits

- Visits allows to make a link between a visit id and an observation time, the (ra,dec) of the telescope, the observation time and observation conditions such airmass, exposure time.
- This allow us to make a selection of visits for which we can select the sources in the Tract-Patch selected

In [ ]:
datasettype = "visitTable"

In [ ]:
print(butler.registry.getDatasetType(datasettype).dimensions)
print(where_clause_instrument)

In [ ]:
dataset_refs = list(
    butler.registry.queryDatasets(
        datasettype, collections=collection_validation, where=where_clause_instrument
    )
)
# Récupère un des refs valides
Nrefs = len(dataset_refs)
print(f"Number of visitTables : {Nrefs}")
ref = dataset_refs[0]
t = butler.get(ref)
Nvis = len(t)
# del t
# gc.collect()
print(f"Total Number of visits {Nvis}")

In [ ]:
if FLAG_DUMP_VISITTABLEECOLUMNS:
    t_columns = list(butler.get(ref).columns)
    print(t_columns)

In [ ]:
df_visits = t

In [ ]:
df_visits.head()

In [ ]:
if FLAG_COMPUTE_VISITSTRACTSPATCHS:
    # Takes Very long time to compute.
    # It is better to filter out visits that does not belong to the selected tract patch as shown below
    df_visits_tractspatches = assign_tract_patch(df_visits, skymap)

### select visits in selected Tract and Patch 

- just need visits ra and dec coordinates to filter those visits in the tract-patch.

In [ ]:
df_visits_in_patch = select_visits_in_patch(df_visits, tract_info, patch_info)

In [ ]:
df_visits_in_patch

### Keep the list in memory

In [ ]:
visitLists_sel = df_visits_in_patch.visitId.values

In [ ]:
print(f"Number of selected visits in  ")

## Get the selected sources

In [ ]:
datasettype = "sourceTable_visit"

In [ ]:
Nvis_sel = len(visitLists_sel)
print(f"Number of visits {Nvis_sel} in tract {tractNbSel} and patch {patchNbSel}")
print(visitLists_sel)

In [ ]:
visit_sel = visitLists_sel[0]
butler.registry.expandDataId(instrument=instrument, visit=visit_sel)

In [ ]:
def load_sourceTables_for_visits(butler, datasettype, instrument, visit_ids):
    dfs = []
    for visit in visit_ids:
        refs = list(
            butler.registry.queryDatasets(
                datasetType=datasettype, dataId={"instrument": instrument, "visit": visit}, findFirst=True
            )
        )
        if refs:
            df = butler.get(refs[0])
            df["visit"] = visit
            dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    else:
        return pd.DataFrame()

In [ ]:
# dt = butler.registry.getDatasetType("src")
# print(dt.dimensions)

In [ ]:
# crash
# print(f"Fetch the {datasettype} for each visit in the list")
# df_all_sources = load_sourceTables_for_visits(butler, datasettype,instrument, visitLists_sel)

In [ ]:
# df_all_sources

In [ ]:
def save_sources_for_visit(butler, datasettype, visit, instrument, output_dir):
    all_sources = []

    try:
        refs = list(
            butler.registry.queryDatasets(
                datasetType=datasettype, dataId={"instrument": instrument, "visit": visit}, findFirst=True
            )
        )
        if refs:
            df = butler.get(refs[0])
            df["visit"] = visit

        all_sources.append(df)

    except Exception as inst:
        print(type(inst))  # the exception type
        print(inst.args)  # arguments stored in .args
        return

    if all_sources:
        # Fusionner toutes les sources
        sources_df = pd.concat(all_sources, ignore_index=True)

        # clean columns with Nans
        sources_df.dropna(axis=1, how="all", inplace=True)
        # keep primary source
        sources_df = sources_df[sources_df.parentSourceId == 0]

        # cut band sources in low SNR
        sources_df["calibSNR"] = sources_df["calibFlux"] / sources_df["calibFluxErr"]
        sources_df = sources_df[sources_df["calibSNR"] >= 5]

        # compute Magnitude
        sources_df["calibMag"] = nJy_to_ab_mag(sources_df["calibFlux"])
        sources_df["calibMagErr"] = nJy_err_to_ab_err(sources_df["calibFlux"], sources_df["calibFluxErr"])

        # drop rows-sources with zero mag
        sources_df = sources_df[sources_df["calibMag"] != 0].dropna(subset=["calibMag"])

        # Sauvegarder dans un fichier Parquet
        file_path = f"{output_dir}/src_visit_{visit}.parquet"
        sources_df.to_parquet(file_path, index=False)
        print(f"Sources for visit {visit} saved to {file_path}")
        return file_path
    else:
        print(f"No sources found for visit {visit}")

### Manage output sources in parquet file

In [ ]:
outputdir_sources = f"output_{instrument}_field{keyfield}_t{tractNbSel}_p{patchNbSel}_{datasettype}"
os.makedirs(outputdir_sources, exist_ok=True)

In [ ]:
all_outputfiles = []
for visit in visitLists_sel:
    outputfile = save_sources_for_visit(butler, datasettype, visit, instrument, outputdir_sources)
    all_outputfiles.append(outputfile)

In [ ]:
outputfile = all_outputfiles[0]
df = pd.read_parquet(outputfile)

In [ ]:
print(list(df.columns))